# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dubai,25.2582,55.3047,71.53,49,0,3.44,AE,1702503141
1,1,sitia,35.2000,26.0986,66.97,84,4,10.94,GR,1702503268
2,2,hithadhoo,-0.6000,73.0833,82.08,77,83,6.38,MV,1702503268
3,3,hermanus,-34.4187,19.2345,61.21,73,100,8.84,ZA,1702503268
4,4,ouarzazat,30.9189,-6.8934,61.09,33,20,6.91,MA,1702503269
...,...,...,...,...,...,...,...,...,...,...
590,590,cotacachi,0.3000,-78.2667,69.35,77,92,3.89,EC,1702503367
591,591,santa elena de uairen,4.6167,-61.1333,74.19,74,100,2.89,VE,1702503367
592,592,arkhangel'sk,64.5401,40.5433,14.13,85,100,6.71,RU,1702503367
593,593,nampula,-15.1165,39.2666,80.55,57,0,4.61,MZ,1702503367


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_df = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                      scale=0.5,
                                      color='Humidity',
                                      hover_cols=['City'],
                                      tiles='OSM',
                                      frame_width=700,
                                      frame_height=500)

# Display the map
humidity_df

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 60
ideal_temp_max = 95
ideal_humidity_min = 50
ideal_humidity_max = 80

ideal_weather = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temp_min) & 
    (city_data_df['Max Temp'] <= ideal_temp_max) & 
    (city_data_df['Humidity'] >= ideal_humidity_min) & 
    (city_data_df['Humidity'] <= ideal_humidity_max)]

# Drop any rows with null values
ideal_weather = city_data_df.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dubai,25.2582,55.3047,71.53,49,0,3.44,AE,1702503141
1,1,sitia,35.2000,26.0986,66.97,84,4,10.94,GR,1702503268
2,2,hithadhoo,-0.6000,73.0833,82.08,77,83,6.38,MV,1702503268
3,3,hermanus,-34.4187,19.2345,61.21,73,100,8.84,ZA,1702503268
4,4,ouarzazat,30.9189,-6.8934,61.09,33,20,6.91,MA,1702503269


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,dubai,AE,25.2582,55.3047,49,
1,sitia,GR,35.2000,26.0986,84,
2,hithadhoo,MV,-0.6000,73.0833,77,
3,hermanus,ZA,-34.4187,19.2345,73,
4,ouarzazat,MA,30.9189,-6.8934,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key, 
    'categories': 'accomodation.hotel',
    'limit': 20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    print(response)
# Display sample data
hotel_df

Starting hotel search
dubai - nearest hotel: No hotel found
<Response [400]>
sitia - nearest hotel: No hotel found
<Response [400]>
hithadhoo - nearest hotel: No hotel found
<Response [400]>
hermanus - nearest hotel: No hotel found
<Response [400]>
ouarzazat - nearest hotel: No hotel found
<Response [400]>


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

In [ ]:
## Notes: I keep getting "Starting hotel search dubai - nearest hotel: No hotel found <Response [400]>" and so on and so forth when I run this code. I was working on this with my TA and we never figured out what the problem was. Since I am not getting any hotels I could not finish the last part of the assignment.